# Scenario:
    1. Split dataframe two 2 dataframe.
    2. capture cust details in DF1
    3. Capture title based stat in Df2
    4. Join both dataframe

In [5]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [7]:
if __name__ == "__main__" :
    spark=(SparkSession
           .builder
           .appName("Prg5_DF_joins_ops")
           .getOrCreate()
          )

In [8]:
parquet_dataset="/home/hari/Downloads/LearningSparkV2-master/parquet_files/userdata1.parquet"

In [13]:
df=spark.read.format("parquet").load(parquet_dataset)
df.show(5)
df.printSchema()

+-------------------+---+----------+---------+--------------------+------+--------------+----------------+------------+---------+---------+--------------------+--------+
|  registration_dttm| id|first_name|last_name|               email|gender|    ip_address|              cc|     country|birthdate|   salary|               title|comments|
+-------------------+---+----------+---------+--------------------+------+--------------+----------------+------------+---------+---------+--------------------+--------+
|2016-02-03 13:25:29|  1|    Amanda|   Jordan|    ajordan0@com.com|Female|   1.197.201.2|6759521864920116|   Indonesia| 3/8/1971| 49756.53|    Internal Auditor|   1E+02|
|2016-02-03 22:34:03|  2|    Albert|  Freeman|     afreeman1@is.gd|  Male|218.111.175.34|                |      Canada|1/16/1968|150280.17|       Accountant IV|        |
|2016-02-03 06:39:31|  3|    Evelyn|   Morgan|emorgan2@altervis...|Female|  7.161.136.94|6767119071901597|      Russia| 2/1/1960|144972.51| Structural

In [67]:
#DF - customer details

df_cust=(df.withColumn("age",F.round((F.datediff(F.current_date(),F.to_date(F.col("birthdate"),format="M/d/yyyy")))/365,2))
         .select(F.col("ID").alias("cust_id"),"title","first_name","last_name","salary","gender",F.col("birthdate").alias("dob"),"age")
         .where(F.trim("title")!='')
         .orderBy("ID")
        )

df_cust.show(2)

+-------+----------------+----------+---------+---------+------+---------+-----+
|cust_id|           title|first_name|last_name|   salary|gender|      dob|  age|
+-------+----------------+----------+---------+---------+------+---------+-----+
|      1|Internal Auditor|    Amanda|   Jordan| 49756.53|Female| 3/8/1971| 52.2|
|      2|   Accountant IV|    Albert|  Freeman|150280.17|  Male|1/16/1968|55.34|
+-------+----------------+----------+---------+---------+------+---------+-----+
only showing top 2 rows



In [60]:
#DF - title wise salary stats

spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
df_sal_1=(df_cust.withColumn("age",F.round((F.datediff(F.current_date(),F.to_date(F.col("dob"),format="M/d/yyyy")))/365,2))
        .select("title","salary","age")
        #.show(5)
        #.withColumn("dob_c",F.to_date("dob",format="M/d/yyyy"))
)
df_sal_1.show(3)
df_sal_stat=(df_sal_1.groupBy("title")
         .agg((F.round(F.avg("age"),2)).alias("avg_age"),F.round(F.avg("salary"),2).alias("avg_salary"))
         .orderBy("title")
)
df_sal_stat.show(3)

+-------------------+---------+-----+
|              title|   salary|  age|
+-------------------+---------+-----+
|   Internal Auditor| 49756.53| 52.2|
|      Accountant IV|150280.17|55.34|
|Structural Engineer|144972.51| 63.3|
+-------------------+---------+-----+
only showing top 3 rows

+-------------------+-------+----------+
|              title|avg_age|avg_salary|
+-------------------+-------+----------+
|           61871.37|   null|      null|
|Account Coordinator|  39.99|  84567.46|
|  Account Executive|   45.6| 110939.16|
+-------------------+-------+----------+
only showing top 3 rows



In [71]:
# joining both DF 

df_join=(df_cust.join(df_sal_stat,df_cust.title==df_sal_stat.title,"inner")
 
)

(df_cust.join(df_sal_stat,df_cust.title==df_sal_stat.title,"full")
 .show(3)
)

+-------+-------------------+----------+---------+---------+------+--------------------+-----+-------------------+-------+----------+
|cust_id|              title|first_name|last_name|   salary|gender|                 dob|  age|              title|avg_age|avg_salary|
+-------+-------------------+----------+---------+---------+------+--------------------+-----+-------------------+-------+----------+
|    588|           61871.37|  Benjamin|  Wheeler|     null|  Male| Saint Eustatius ...| null|           61871.37|   null|      null|
|    340|Account Coordinator|   Stephen|    Perry| 83759.68|  Male|            3/2/1986|37.21|Account Coordinator|  39.99|  84567.46|
|    453|Account Coordinator|    Jeremy|  Bennett|193301.99|  Male|           1/19/1967|56.33|Account Coordinator|  39.99|  84567.46|
+-------+-------------------+----------+---------+---------+------+--------------------+-----+-------------------+-------+----------+
only showing top 3 rows



In [86]:
#Union
print(df_join.count())
df_join_male=(df_join.where(F.col("gender")=='Male'))
print(df_join_male.count())
df_join_female=(df_join.where(F.col("gender")=='Female'))
print(df_join_female.count())

#Union-Merge of 2 DF

df_join_2=df_join_male.union(df_join_female)
print(df_join_2.count())

print((df_join.where((F.col("gender")!='Male') & (F.col("gender")!='Female'))).count())

803
360
388
748
55


23/05/07 19:06:42 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1306406 ms exceeds timeout 120000 ms
23/05/07 19:06:42 WARN SparkContext: Killing executors is not supported by current scheduler.
